### Imports


In [17]:
import pandas as pd
from scipy.stats import zscore, skew
import numpy as np
from typing import List
import ast

### Defining important methods

In [18]:
def print_initial_df(df: pd.DataFrame) -> None:
    print("Initial DataFrame:\n")
    print(df.head())  # Print only the first 5 rows

def count_nulls_in_columns(df: pd.DataFrame) -> None:

    null_counts = df.isnull().sum()  
    total_rows = len(df)  
    print("Percentage of null values in each column:")
    for column, null_count in null_counts.items():
        # Calculate percentage of null values for each column
        null_percentage = (null_count / total_rows) * 100
        print(f"Column: '{column}' has {null_percentage:.2f}% null values.")

def estimate_end_year(row) -> int:
    if np.isnan(row['endYear']):  
        if row['startYear'] >= 2020:  # Recent shows, assume still ongoing
            return 2022  
        else:
            return row['startYear'] + 2  # Assume average duration of 2 years
    return row['endYear'] 

def null_percentage(df: pd.DataFrame, column_name: str) -> float:
    if column_name not in df.columns:
        raise ValueError(f"Column '{column_name}' not found in the DataFrame.")
    
    total_rows = len(df)
    null_count = df[column_name].isna().sum()
    percentage = (null_count / total_rows) * 100

    return round(percentage, 2) 

def get_unique_actors(df: pd.DataFrame) -> set:
    # Safely evaluate the 'cast' column and handle potential errors
    def safe_eval(x):
        try:
            return eval(x) if isinstance(x, str) else []
        except Exception as e:
            print(f"Error evaluating: {x}, error: {e}")
            return []

    df['cast'] = df['cast'].apply(safe_eval)  # Convert string lists to actual lists
    unique_actors = set(actor for cast_list in df['cast'] for actor in cast_list)
    return unique_actors

def get_unique_genres(df: pd.DataFrame) -> list:
    # Initialize an empty set to store unique genres
    unique_genres = set()
    
    for genres in df['genres']:
        if isinstance(genres, str):  # Check if the value is a string
            # Split the string by commas and remove extra spaces, then add to the set
            genres_list = [genre.strip() for genre in genres.split(',')]
            unique_genres.update(genres_list)  # Add genres to the set
    
    return list(unique_genres)

### Initial dataframe

In [19]:
movies_df = pd.read_csv("netflix_list.csv", index_col=0)
print_initial_df(movies_df.head())

Initial DataFrame:

                         title popular_rank certificate  startYear  endYear  \
imdb_id                                                                       
tt4052886              Lucifer            1          15     2016.0      NaN   
tt0993840     Army of the Dead            2          18     2021.0      NaN   
tt7255502  The Kominsky Method            3          18     2018.0   2021.0   
tt0108778              Friends            4         13+     1994.0   2004.0   
tt9251798             Ragnarok            5          18     2020.0      NaN   

           episodes runtime      type  orign_country   language  \
imdb_id                                                           
tt4052886      93.0      42  tvSeries  United States    English   
tt0993840       NaN     148     movie  United States    English   
tt7255502      22.0      30  tvSeries  United States    English   
tt0108778     235.0      22  tvSeries  United States    English   
tt9251798      12.0     

### Make copy of the initial dataset

In [20]:
movies_df_copy = movies_df.copy()
print("Copy of the DataFrame:\n")
print(movies_df_copy.head()) 

Copy of the DataFrame:

                         title popular_rank certificate  startYear  endYear  \
imdb_id                                                                       
tt4052886              Lucifer            1          15     2016.0      NaN   
tt0993840     Army of the Dead            2          18     2021.0      NaN   
tt7255502  The Kominsky Method            3          18     2018.0   2021.0   
tt0108778              Friends            4         13+     1994.0   2004.0   
tt9251798             Ragnarok            5          18     2020.0      NaN   

           episodes runtime      type  orign_country   language  \
imdb_id                                                           
tt4052886      93.0      42  tvSeries  United States    English   
tt0993840       NaN     148     movie  United States    English   
tt7255502      22.0      30  tvSeries  United States    English   
tt0108778     235.0      22  tvSeries  United States    English   
tt9251798      12.0 

### Check percentage of null values in all the columns
I want to check the null values in every column

In [21]:
count_nulls_in_columns(movies_df_copy)

Percentage of null values in each column:
Column: 'title' has 0.00% null values.
Column: 'popular_rank' has 0.00% null values.
Column: 'certificate' has 65.62% null values.
Column: 'startYear' has 7.35% null values.
Column: 'endYear' has 83.93% null values.
Column: 'episodes' has 67.12% null values.
Column: 'runtime' has 0.03% null values.
Column: 'type' has 0.03% null values.
Column: 'orign_country' has 0.00% null values.
Column: 'language' has 0.19% null values.
Column: 'plot' has 5.69% null values.
Column: 'summary' has 0.00% null values.
Column: 'rating' has 14.38% null values.
Column: 'numVotes' has 14.38% null values.
Column: 'genres' has 0.03% null values.
Column: 'isAdult' has 0.00% null values.
Column: 'cast' has 0.00% null values.
Column: 'image_url' has 0.00% null values.


### Handling Columns with Missing Values or Columns that do not bring any insight to predict the rating
certificate: Removed due to a high number of missing values and its low relevance to predicting ratings based on votes.

endYear and episodes: Retained for potential value in prediction but will be imputed later if possible to handle missing data.

image_url: Removed as it does not provide useful information for predicting ratings.

title: The title does not bring any value to predict rating

runtime: The runtime is not explained in the dataset so I do not know what it is about so i decided to drop it

plot and summary: The plot and summary do not bring any value to predict rating

imdb_id: The unique identifier does not bring any value to predict rating

In [22]:
movies_df_copy = movies_df_copy.reset_index(drop=True)
movies_df_copy = movies_df_copy.reset_index(drop=True).drop(columns=['certificate', 'image_url', 'title', 'runtime', 'plot', 'summary'])
print(movies_df_copy.head())

  popular_rank  startYear  endYear  episodes      type  orign_country  \
0            1     2016.0      NaN      93.0  tvSeries  United States   
1            2     2021.0      NaN       NaN     movie  United States   
2            3     2018.0   2021.0      22.0  tvSeries  United States   
3            4     1994.0   2004.0     235.0  tvSeries  United States   
4            5     2020.0      NaN      12.0  tvSeries         Norway   

    language  rating  numVotes                genres  isAdult  \
0    English     8.1  250884.0   Crime,Drama,Fantasy        0   
1    English     5.8  110780.0   Action,Crime,Horror        0   
2    English     8.2   28795.0          Comedy,Drama        0   
3    English     8.9  861843.0        Comedy,Romance        0   
4  Norwegian     7.5   26606.0  Action,Drama,Fantasy        0   

                                                cast  
0  ['Tom Ellis', 'Lauren German', 'Lesley-Ann Bra...  
1  ['Dave Bautista', 'Ella Purnell', 'Ana de la R...  
2  ['

### Check unique actors in cast

Before deciding what to do with the cast column I want to check how many unique values it has

In [23]:
unique_actors = get_unique_actors(movies_df_copy)
print(f"Total unique actors: {len(unique_actors)}")
print(list(unique_actors))

Error evaluating: -, error: invalid syntax (<string>, line 1)
Error evaluating: -, error: invalid syntax (<string>, line 1)
Error evaluating: -, error: invalid syntax (<string>, line 1)
Error evaluating: -, error: invalid syntax (<string>, line 1)
Error evaluating: -, error: invalid syntax (<string>, line 1)
Error evaluating: -, error: invalid syntax (<string>, line 1)
Error evaluating: -, error: invalid syntax (<string>, line 1)
Error evaluating: -, error: invalid syntax (<string>, line 1)
Error evaluating: -, error: invalid syntax (<string>, line 1)
Error evaluating: -, error: invalid syntax (<string>, line 1)
Error evaluating: -, error: invalid syntax (<string>, line 1)
Error evaluating: -, error: invalid syntax (<string>, line 1)
Error evaluating: -, error: invalid syntax (<string>, line 1)
Error evaluating: -, error: invalid syntax (<string>, line 1)
Error evaluating: -, error: invalid syntax (<string>, line 1)
Error evaluating: -, error: invalid syntax (<string>, line 1)
Error ev

### Dropping cast column

The "cast" column in the dataset contains a list of the first and last names of all significant actors featured in each movie or series. Given that this column represents categorical, non-numerical data with a high degree of granularity (comprising 58,151 unique actors), converting this feature into numerical values for correlation analysis would introduce significant complexity and may not provide meaningful insights for this particular analysis.

Therefore, in order to streamline the dataset and focus on the more relevant numerical features that can directly contribute to the correlation with movie ratings, the "cast" column will be excluded from the analysis. This decision ensures that the correlation analysis remains focused on the quantitative aspects of the dataset, facilitating more efficient and interpretable results.

In [24]:
movies_df_copy = movies_df_copy.drop(columns=['cast'])
print(movies_df_copy)

     popular_rank  startYear  endYear  episodes       type  orign_country  \
0               1     2016.0      NaN      93.0   tvSeries  United States   
1               2     2021.0      NaN       NaN      movie  United States   
2               3     2018.0   2021.0      22.0   tvSeries  United States   
3               4     1994.0   2004.0     235.0   tvSeries  United States   
4               5     2020.0      NaN      12.0   tvSeries         Norway   
...           ...        ...      ...       ...        ...            ...   
7003       16,543     2019.0      NaN       NaN  tvEpisode  United States   
7004       16,569     2019.0      NaN       NaN  tvEpisode              -   
7005       16,599     2019.0      NaN       NaN  tvEpisode              -   
7006       16,627     2021.0      NaN       NaN  tvEpisode  United States   
7007       16,648     2020.0      NaN       NaN  tvEpisode  United States   

       language  rating  numVotes                genres  isAdult  
0       

### Checking again what columns have a null values

In [25]:
count_nulls_in_columns(movies_df_copy)

Percentage of null values in each column:
Column: 'popular_rank' has 0.00% null values.
Column: 'startYear' has 7.35% null values.
Column: 'endYear' has 83.93% null values.
Column: 'episodes' has 67.12% null values.
Column: 'type' has 0.03% null values.
Column: 'orign_country' has 0.00% null values.
Column: 'language' has 0.19% null values.
Column: 'rating' has 14.38% null values.
Column: 'numVotes' has 14.38% null values.
Column: 'genres' has 0.03% null values.
Column: 'isAdult' has 0.00% null values.


### Analyzing the Relationship Between startYear and endYear
This code provides a statistical summary (count, mean, std, min, max) of the startYear and endYear columns, focusing on non-missing values. It helps understand the typical range, central tendencies and variability in these columns.

In [26]:
movies_df_copy[['startYear', 'endYear']].dropna().describe()

,startYear,endYear
count,1126.000000,1126.000000
mean,2014.974245,2016.613677
std,6.255895,5.195806
min,1966.000000,1969.000000
25%,2014.000000,2016.000000
50%,2017.000000,2018.000000
75%,2019.000000,2019.000000
max,2021.000000,2022.000000


## Explanation for Estimating Missing endYear Values
The missing values in the endYear column are being estimated based on the patterns observed in the dataset. The dataset provides insights into the distribution of both startYear and endYear, which allows us to make informed assumptions about how to estimate the missing values.

### Data Overview:
The startYear ranges from 1966 to 2021, with a mean of approximately 2014.97 and a median of 2017.
The endYear ranges from 1969 to 2022, with a mean of 2016.61 and a median of 2018.
A significant number of the startYear values are between 2014 and 2019, with the majority of the endYear values clustered around 2016 to 2019.
Assumptions for Estimating Missing endYear Values:

### Recent Shows (Start Year ≥ 2020):
Shows that started in 2020 or later are likely finished, as we are currently in 2025.
Assumption: For these shows, the missing endYear values are estimated as 2022, which is the most recent year in the dataset.

### Older Shows (Start Year < 2020):
For shows that started before 2020, the general pattern shows that they tend to have a lifespan of around 1-3 years.
Assumption: For these shows, we estimate the endYear to be approximately startYear + 2, assuming an average duration of 2 years. This reflects the typical life cycle of many older shows, which often had shorter or more limited runs.

### Why 2 Years for Older Shows?
Data-Driven Decision: Based on the dataset, the majority of the endYear values occur within 1-3 years of the startYear, indicating that many shows in the dataset had relatively short lifespans.

Balanced Approach: A 2-year duration is a reasonable estimate based on the observed data, providing a balanced approach that avoids underestimating the duration of shows that last longer than 1 year while also not overestimating the duration of shows with shorter runs.

In [27]:
movies_df_copy['endYear'] = movies_df_copy.apply(estimate_end_year, axis=1)
percent_null = null_percentage(movies_df_copy, "endYear")
print(f"Percentage of null values in 'endYear': {percent_null}%")

Percentage of null values in 'endYear': 7.35%


### Checking again how many null values each column has

In [28]:
count_nulls_in_columns(movies_df_copy)

Percentage of null values in each column:
Column: 'popular_rank' has 0.00% null values.
Column: 'startYear' has 7.35% null values.
Column: 'endYear' has 7.35% null values.
Column: 'episodes' has 67.12% null values.
Column: 'type' has 0.03% null values.
Column: 'orign_country' has 0.00% null values.
Column: 'language' has 0.19% null values.
Column: 'rating' has 14.38% null values.
Column: 'numVotes' has 14.38% null values.
Column: 'genres' has 0.03% null values.
Column: 'isAdult' has 0.00% null values.


### Check the type of numVotes column
Since only 14% of the values of this column are null I will treat tream them. But first I want to check the type of this column to decide how I am going to replace these values

In [29]:
print("The type of the numVotes column is " + str(movies_df_copy["numVotes"].dtype))

The type of the numVotes column is float64


### Convert column type to integer and delete float values

In [30]:
# Filter the 'numVotes' column to exclude NaN values and identify values with decimals
num_votes_with_decimals = movies_df_copy[movies_df_copy["numVotes"].notna() & (movies_df_copy["numVotes"] % 1 != 0)]

# Count the number of non-null decimal values in 'numVotes'
decimal_count = num_votes_with_decimals.shape[0]

# Count the number of zero values (excluding NaNs) in 'numVotes'. Important to check so later I do not confuse them with Nan values when I convert it to an integer column
zero_count = len(movies_df_copy[movies_df_copy["numVotes"].notna() & (movies_df_copy["numVotes"] == 0)])

print("The amount of decimal values in 'numVotes' is: " + str(decimal_count))
print("The amount of zero values in 'numVotes' before filtering is: " + str(zero_count))

The amount of decimal values in 'numVotes' is: 0
The amount of zero values in 'numVotes' before filtering is: 0


### Change column type of numVotes to integer
Since there are no decimal values in this column, I am going to change the type of the column to integer

In [31]:
# Replace NaN values with 0 and convert the column to integers
movies_df_copy["numVotes"] = movies_df_copy["numVotes"].fillna(0).astype(int)

print(movies_df_copy["numVotes"].head())


0    250884
1    110780
2     28795
3    861843
4     26606
Name: numVotes, dtype: int64


### Checking skewness in the numVotes column
Before taking a decision on how I will replace the null values, I want to understand the skewness of the column

In [32]:
# Filter out 0 values before calculating skewness since they are missing values and they do not bring any insight to the skewness
non_zero_votes = movies_df_copy[movies_df_copy['numVotes'] > 0]['numVotes']

# Calculate skewness only for non-zero values
skewness_value = skew(non_zero_votes)
print("Skewness (Ignoring Nan):", skewness_value)

Skewness (Ignoring Nan): 10.037454384886306


### Understanding the skewness result in numVotes column
The numVotes column is highly right-skewed (10.04), meaning most shows have very few votes, while a few popular ones have extremely high votes. Therefore, I decide to fill up the null values of this column using the Random Forest algorithm because it can learn patterns from other features in the dataset.

To use the mean to fill up these null values would not be intelligent because it is sensitive to outliers(This column is highly skewed). The imputed values would probably be biased toward the central tendency, which does not reflect the actual patterns of the data. 

To use the median imputation would make more since it is less sensitive to outliers and it is a better choice when data is skewed. However, it still ignores the relationship between numVotes and other features of the dataset

First thing i need to check is the correlation between every feature and the correlation of every feature with the numVotes column(the target)


In [33]:
# Select only numeric columns
numeric_columns = movies_df_copy.select_dtypes(include=['number']).columns

# Remove 'numVotes' from the numeric columns for correlation calculation
numeric_columns = numeric_columns[numeric_columns != 'numVotes']

# Drop rows with missing values in the entire dataset (to include 'numVotes' as well)
movies_df_copy_clean = movies_df_copy.dropna(subset=['numVotes'])

# Calculate correlation only for numeric columns excluding 'numVotes'
corr_with_target = movies_df_copy_clean[numeric_columns].corrwith(movies_df_copy_clean['numVotes']).sort_values(ascending=False)

# Display the top 5 most correlated features with 'numVotes'
print("Most correlated features with numVotes:")
print(corr_with_target.head())




Most correlated features with numVotes:
rating       0.165060
episodes     0.050971
endYear     -0.159264
startYear   -0.185714
isAdult           NaN
dtype: float64


c:\Users\nicol\Desktop\DS practice\.venv\lib\site-packages\numpy\lib\_function_base_impl.py:3045: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\nicol\Desktop\DS practice\.venv\lib\site-packages\numpy\lib\_function_base_impl.py:3046: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
